<img class="img-responsive center-block" src="https://upload.wikimedia.org/wikipedia/commons/7/79/Docker_%28container_engine%29_logo.png">
<h1 class="text-right">Mini taller<h1>


## Contenido

- Docker
- Instalación
- Componentes de docker
- Dockerizar mi aplicación

## Linux

- Versión del kernel minima requerida: 3.10

- Agregar clave gpg

In [ ]:
sudo apt-key adv --keyserver hkp://p80.pool.sks-keyservers.net:80 --recv-keys 58118E89F3A912897C070ADBF76221572C52609D

- Agrega el repositorio de Docker para apt

In [ ]:
echo "deb https://apt.dockerproject.org/repo ubuntu-trusty main" > /etc/apt/sources.list.d/docker.list

- (extra) Instalar drivers extra para AUF

In [ ]:
sudo apt-get update
sudo apt-get install linux-image-extra-$(uname -r)

- Instalar Docker

In [ ]:
sudo apt-get install docker-engine

- Si es requerido iniciar el servicio

In [ ]:
sudo service docker start

- Ser parte del grupo de Docker, (ejecutar docker sin ser super user)

In [ ]:
sudo usermod -aG docker <usuario>

## Mac and Windows

- Docker Toolbox - Compatible with Mac OS X 10.8+ and Windows 7+
 

## Imagenes, contenedores y otras bestias

In [1]:
docker version

Client:
 Version:      1.9.1
 API version:  1.21
 Go version:   go1.4.2
 Git commit:   a34a1d5
 Built:        Fri Nov 20 13:12:04 UTC 2015
 OS/Arch:      linux/amd64

Server:
 Version:      1.9.1
 API version:  1.21
 Go version:   go1.4.2
 Git commit:   a34a1d5
 Built:        Fri Nov 20 13:12:04 UTC 2015
 OS/Arch:      linux/amd64


### Images

Es un snapshot de las dependencias, pasos a realizar y utilerias de una aplicación.

Tres formas de obtener imagenes:
  - [Repositorio](https://hub.docker.com/)
  - Construirlo mediante Dockerfile
  - Artesanal - (Manual)
  
  
### Container

Es una instancia de la imagen, lleva acabo la ejecución de la aplicación aislada del mundo a excepción que especifiquemos recursos compartidos o puntos de conexión.

  - **Binding**: Ligar un archivo local al contenedor
  - **Volumen**: Un contenedor actua almacenamiento
  - **Port binding**: Asociar un puerto del host al contenedor.

## Dockerizar mi aplicación

- Busqueda y descarga de imagen base.
- Levantar un contenedor.
- Constuir la imagen para nuestra app
- Exponer la aplicación al mundo
- Enlazar la base de datos

In [3]:
docker search --stars=10 alpine

NAME                         DESCRIPTION                                     STARS     OFFICIAL   AUTOMATED
alpine                       A minimal Docker image based on Alpine Lin...   427       [OK]       
frolvlad/alpine-oraclejdk8   The smallest Docker image with OracleJDK 8...   40                   [OK]
anapsix/alpine-java          Oracle Java8 with GLIBC 2.21 over AlpineLinux   19                   [OK]


In [34]:
docker pull alpine

Using default tag: latest
latest: Pulling from library/alpine
Digest: sha256:78a756d480bcbc35db6dcc05b08228a39b32c2b2c7e02336a2dcaa196547a41d
Status: Downloaded newer image for alpine:latest


In [16]:
cat > Dockerfile <<- END
FROM alpine
MAINTAINER "Miguel Angel Gordian"

WORKDIR /home/app
ADD . /home/app

# RUN adduser -D app
RUN apk update && \
    apk add py-pip && \
    pip install -r requirements.txt

# USER app

EXPOSE 80

ENTRYPOINT python index.py
END

cat Dockerfile

FROM alpine
MAINTAINER "Miguel Angel Gordian"

WORKDIR /home/app
ADD . /home/app

# RUN adduser -D app
RUN apk update &&     apk add py-pip &&     pip install -r requirements.txt

# USER app

EXPOSE 80

ENTRYPOINT python index.py


In [17]:
docker build -t superapp .


Step 1 : FROM alpine
 ---> 74e49af2062e
Step 2 : MAINTAINER "Miguel Angel Gordian"
 ---> Using cache
 ---> 45216bf2d6ad
Step 3 : WORKDIR /home/app
 ---> Using cache
 ---> 081a21a3dd3d
Step 4 : ADD . /home/app
 ---> 7a6b6340b39c
Removing intermediate container d99d763bd80e
Step 5 : RUN apk update &&     apk add py-pip &&     pip install -r requirements.txt
 ---> Running in b4e3e32eb523
fetch http://dl-4.alpinelinux.org/alpine/v3.3/main/x86_64/APKINDEX.tar.gz
fetch http://dl-4.alpinelinux.org/alpine/v3.3/community/x86_64/APKINDEX.tar.gz
v3.3.1-2-g6c5f200 [http://dl-4.alpinelinux.org/alpine/v3.3/main]
v3.3.0_rc3-205-g8864778 [http://dl-4.alpinelinux.org/alpine/v3.3/community]
OK: 5854 distinct packages available
(1/12) Installing libbz2 (1.0.6-r4)
(2/12) Installing expat (2.1.0-r2)
(3/12) Installing libffi (3.2.1-r1)
(4/12) Installing gdbm (1.11-r1)
(5/12) Installing ncurses-terminfo-base (6.0-r6)
(6/12) Installing ncurses-terminfo (6.0-r6)
(7/12) Installing ncurses-libs (6.0-r6)
(8/12) 

In [18]:
docker rm -f superapp_1
docker run -d --name superapp_1 superapp

superapp_1
ecdf4d68227ae60150092890c91f3f47217b08a735f31f68edd87118b35a4b48


In [33]:
docker ps

CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS                PORTS                      NAMES
ecdf4d68227a        superapp            "/bin/sh -c 'python i"   10 minutes ago      Up 10 minutes         80/tcp                     superapp_1
379013fdbb35        alpine              "top"                    2 hours ago         Up 2 hours (Paused)                              suspicious_sinoussi
fe81324fbb4e        postgres            "/docker-entrypoint.s"   44 hours ago        Up 2 hours            127.0.0.1:5432->5432/tcp   postgres


In [28]:
docker logs superapp_1

 * Running on http://0.0.0.0:80/ (Press CTRL+C to quit)


In [39]:
# check
IP=`docker inspect -f "{{ .NetworkSettings.IPAddress }}" superapp_1`
echo "La IP asignada al contenedor superapp: $IP"
firefox $IP

La IP asignada al contenedor superapp: 172.17.0.4


In [46]:
echo "--> Elimina si existe una instancia de supeapp_2"
docker rm -f superapp_2

echo "--> Inicia contenedor con comunicación hacia el exterior"
docker run -d -p 0.0.0.0:80:80 --name superapp_2 superapp

echo "--> Puertos externos asociados al contenedor"
IP_PORT=`docker port superapp_2 80`
echo $IP_PORT
firefox $IP_PORT

--> Elimina si existe una instancia de supeapp_2
superapp_2
--> Inicia contenedor con comunicación hacia el exterior
fdf3e7a421bf8741e0510f3180c37232b8ed84c7a03acc3b42a1cfea0f9ede09
--> Puertos externos asociados al contenedor
0.0.0.0:80


## Recursos

- [Instalación en Mac](https://docs.docker.com/mac/step_one/)
- [Instalación en Windows](https://docs.docker.com/windows/step_one/)
- [Instalacion en Linux](https://docs.docker.com/engine/installation/ubuntulinux/)
- [Docker 101](http://slides.com/miguelangelgordian/docker-101)
- [Manejo de paquetes para alpine](http://wiki.alpinelinux.org/wiki/Alpine_Linux_package_management)